# Dental ED code

This notebook describes the different steps followed in the analysis of the dental ED data. 
It contains:
- data loading
- building the dataset
- exclusion criteria
- data variables transformation: continuous to ordinal variables
- computation of descriptive statistics
- ICC to evaluate the need for multlevel model
- logistic regression analysis

### Library installation

In [ ]:
install.packages("readstata13")
install.packages("lme4")
library(readstata13)
library(lme4)

### Load data
Our input data set extract from SQL is a data frame that contain for each patient and visit the following information:

    •	DateServiceStarted
    •	MemberBirthYear
    •	MemberGender
    •	MemberZipCode
    •	CongestiveHeartFailure
    •	CardiacArrhythmias
    •	ValvularDisease
    •	PulmonaryCirculationDisorders
    •	PeripheralVascDisorders
    •	HypertensionUncomp
    •	HypertensionComp
    •	Paralysis
    •	OtherNeurDisorders
    •	ChronicPulmDisease
    •	DiabetesUncomp
    •	DiabetesComp
    •	Hypothyroidism
    •	RenalFailure
    •	LiverDisease
    •	PUDNoBleeding
    •	HIVAIDS
    •	Lymphoma
    •	MetastaticCancer
    •	SolidTumorNoMet
    •	RACollagenDisease
    •	Coagulopathy
    •	Obesity
    •	WeightLoss
    •	FluidElecDisorder
    •	AnemiaBloodLoss
    •	AnemiaDeficiency
    •	AlcoholAbuse
    •	DrugAbuse
    •	Psychoses
    •	Depression
    •	EDuse
    •	admissions
    •	DentalPrevent
    •	id

In [ ]:
data = dbGetQuery(cn,  " select * from user.dbo.data") #load data prepared in SQL
rucc = read.csv("fipsrucc.csv") #load RUCC score for each county
zipc = read.csv("zipcounty.csv") #load conversion table for zip code and fips code
dentists = read.dta13("NPI.dta") #load locations of dental practices 
zipincome = read.csv("zipincome.csv") #load median household incomes by zip code
fipspop = read.csv("fipspop") #load population by county
eci = read.csv("eci.csv") #load klookup table for ECI scoring

**NOTE that all the csv files imporated in the previous section are available to download in the GitHub repository.** 

### Build dataset

In [ ]:
data$outcome = !is.na(data$DateServiceStarted) #if DateServiceStarted is NA, then the patient never visted the ED for a dental complaint during the study period; returns FALSE
data$outcome_ = ifelse(data$outcome==T,"ED Visit","No ED Visit") #clarify labels for outcome measure
exp$outcome_ = factor(exp$outcome_, levels = c("No ED Visit","ED Visit")) #define factor levels to set No ED Visit as reference
data$income = zipincome$income[match(data$zip,zipincome$zip)] #match median household income to each record based on zip code
data$fips = zipc$fips[match(data$zip,zipc$zip)] #match each patient's zip code from the insurance database with its corresponding fips code
data$rucc = rucc$score[match(data$fips,rucc$fips)] #match RUCC scores based on fips code
dentists$fips = zipc$fips[match(dentists$zip,zipc$zip)] #match each dentist's practice location zip code with its corresponding fips code
dentisttable = data.frame(table(dentists$fips)) #create table with nunber of dentists in each county
colnames(dentisttable) = c("fips","freq") #rename columns
dentisttable$pop = fipspop$pop[match(dentisttable$fips,fipspop$fips)] #match county population based on fips code
dentisttable$density = round((dentisttable$freq/dentisttable$pop)*100000,1) #calculate number of dentists in each county per 100,000 population
data$dentists = dentisttable$density[match(data$fips,dentisttable$fips)] #match dentists density by county
for (t in 5:35){data[,t] = data[,t] * eci[(t-4),2]} #given that columns 5:35 list a 0 or 1 for each comborbidity from the insurance dataset, the following for loop will return a 0 or the proper factor for each condition since the eci table is listed in the same order, transposed starting in row 1 
data$eci = rowSums(data[,5:35]) #add the total of all factors to calculate the eci score for each patient
data$age = 2015 - data$MemberBirthYear #calculate age in years as of the starting point of the study period

### Exclusion criteria
- Missing age and/or gender
- Patients age under 18 years old
- Patients with gender unknown (U)

In [ ]:
#APPLY EXCLUSION CRITERIA
data = data[!is.na(data$age),] #exclude missing age
data = data[data$age>17.99999,] #exclude age under 18 years
data = data[!is.na(data$MemberGender),] #exclude missing gender
data = data[!data$MemberGender=="U",] #exclude unknown gender
data$gender_ = factor(exp$MemberGender, levels = c("F","M")) #define female as the reference level


### Transform continuous variables into ordinal variables

In [ ]:
data$age_ = ifelse(data$age<34.9999,"18-34",ifelse(data$age<49.9999,"35-49",ifelse(data$age<64.9999,"50-64","65+"))) #bin age
data$age_ = factor(data$age_, levels = c("18-34","35-49","50-64","65+")) #set youngest bin as the reference level
data$rucc_ = ifelse(is.na(data$rucc_),NA,ifelse(data$rucc_<3.5,"METRO",ifelse(data$rucc_<6.5,"NONMETRO","RURAL"))) #bin rucc scores as metro, nonmetro, and rural
exp$rucc_ = factor(exp$rucc_, levels = c("METRO","NONMETRO","RURAL")) #set the most urban bin as the reference level
data$median_income_ = ifelse(data$income<as.numeric(summary(data$income)[2]),"Q1",ifelse(data$income<as.numeric(summary(data$income)[3]),"Q2",ifelse(data$income<as.numeric(summary(data$income)[5]),"Q3","Q4"))) #bin median household income into quartiles
data$median_income_ = factor(data$median_income_, levels = c("Q1","Q2","Q3","Q4")) #set the lowest income bin as the reference level
data$dentist_density_ = ifelse(data$dentists<as.numeric(summary(data$dentists)[2]),"Q1",ifelse(data$dentists<as.numeric(summary(data$dentists)[3]),"Q2",ifelse(data$dentists<as.numeric(summary(data$dentists)[5]),"Q3","Q4"))) #bin dentist density into quartiles
data$dentist_density_ = factor(data$dentist_density_, levels = c("Q1","Q2","Q3","Q4")) #set the fewest dentists per population bin as the reference level
data$dental_care_ = ifelse(data$DentalPrevent==0,"0",ifelse(data$DentalPrevent<3.5,"1-3",ifelse(data$DentalPrevent<6.5,"4-6","7+"))) #bin by number of dental care visits
data$dental_care_ = factor(data$dental_care_, levels = c("0","1-3","4-6","7+")) #set zero dental care visits as the reference level
data$hospital_admissions_ = ifelse(is.na(data$admissions),"0","1+") #binary variable for whether the patient had a hospital admission recorded during the study period
data$hospital_admissions_ = factor(data$hospital_admissions_, levels = c("0","1+")) #set zero hospital admissions as the reference level
data$ECI_ = ifelse(data$eci<10,"ECI<10","ECI 10+") #divide eci score into two catagories with 10 as the cutoff 
data$ECI_ = factor(data$ECI_,levels=c("ECI<10","ECI 10+")) #set the lowest comorbidity burden as the reference level

### Descriptive statistics

In [ ]:
datatrunc = select(data,outcome_,age_,gender_,rucc_,median_income_,dentist_density_,dental_care_,hospital_admissions_,ECI_) #create a truncated version of the dataset with only the variables used in the logistic regression 
#the following function, tmaker, creates a table with frequency and percent of each factor level by whether an ED visit was recorded
tmaker = function(x,y) {
  t = data.frame(cbind(table(x,y),table(x,y)/rowSums(table(x,y)),rowSums(table(x,y)),rowSums(table(x,y))/sum(rowSums(table(x,y)))))
  ct = cbind.data.frame(data.frame(row.names(t),paste0(t[,1]," (",round(t[,3]*100,1),"%)")),data.frame(paste0(t[,2]," (",round(t[,4]*100,1),"%)")),data.frame(paste0(t[,5]," (",round(t[,6]*100,1),"%)")))
  colnames(ct) = c("Variable","No ED Visit","ED Visit","Total")
  return(ct)
}
#the following lines of code apply the tmaker function to each variable in the analysis 
a = tmaker(datatrunc[,2],datatrunc$outcome_) 
b = tmaker(datatrunc[,3],datatrunc$outcome_)
c = tmaker(datatrunc[,4],datatrunc$outcome_)
d = tmaker(datatrunc[,5],datatrunc$outcome_)
e = tmaker(datatrunc[,6],datatrunc$outcome_)
f = tmaker(datatrunc[,7],datatrunc$outcome_)
g = tmaker(datatrunc[,8],datatrunc$outcome_)
h = tmaker(datatrunc[,9],datatrunc$outcome_)
desctable = rbind.data.frame(a,b,c,d,e,f,g,h) #combines all tables for each variable into one table
write.csv(aetnatable,"/Users/gregpeters/OneDrive - Harvard University/EM/EM_Manuscripts/Dental_ED/aetna_dental_table-may27.csv") #save descriptive statistics 
paste0(table(data$outcome_)," (",round(table(data$outcome_)/sum(table(data$outcome_))*100,1),"%)") #returns the number of patients and percent in each group
#the following lines of code return the number of missing records and percent for each variable by group
paste0(table(!is.na(data[data$outcome==TRUE,2]))," (",round(table(!is.na(data[data$outcome==TRUE,2]))/sum(table(!is.na(data[data$outcome==TRUE,2])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==TRUE,3]))," (",round(table(!is.na(data[data$outcome==TRUE,3]))/sum(table(!is.na(data[data$outcome==TRUE,3])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==TRUE,4]))," (",round(table(!is.na(data[data$outcome==TRUE,4]))/sum(table(!is.na(data[data$outcome==TRUE,4])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==TRUE,5]))," (",round(table(!is.na(data[data$outcome==TRUE,5]))/sum(table(!is.na(data[data$outcome==TRUE,5])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==TRUE,6]))," (",round(table(!is.na(data[data$outcome==TRUE,6]))/sum(table(!is.na(data[data$outcome==TRUE,6])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==TRUE,7]))," (",round(table(!is.na(data[data$outcome==TRUE,7]))/sum(table(!is.na(data[data$outcome==TRUE,7])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==TRUE,8]))," (",round(table(!is.na(data[data$outcome==TRUE,8]))/sum(table(!is.na(data[data$outcome==TRUE,8])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==TRUE,9]))," (",round(table(!is.na(data[data$outcome==TRUE,9]))/sum(table(!is.na(data[data$outcome==TRUE,9])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==FALSE,2]))," (",round(table(!is.na(data[data$outcome==FALSE,2]))/sum(table(!is.na(data[data$outcome==FALSE,2])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==FALSE,3]))," (",round(table(!is.na(data[data$outcome==FALSE,3]))/sum(table(!is.na(data[data$outcome==FALSE,3])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==FALSE,4]))," (",round(table(!is.na(data[data$outcome==FALSE,4]))/sum(table(!is.na(data[data$outcome==FALSE,4])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==FALSE,5]))," (",round(table(!is.na(data[data$outcome==FALSE,5]))/sum(table(!is.na(data[data$outcome==FALSE,5])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==FALSE,6]))," (",round(table(!is.na(data[data$outcome==FALSE,6]))/sum(table(!is.na(data[data$outcome==FALSE,6])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==FALSE,7]))," (",round(table(!is.na(data[data$outcome==FALSE,7]))/sum(table(!is.na(data[data$outcome==FALSE,7])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==FALSE,8]))," (",round(table(!is.na(data[data$outcome==FALSE,8]))/sum(table(!is.na(data[data$outcome==FALSE,8])))*100,1),"%)")
paste0(table(!is.na(data[data$outcome==FALSE,9]))," (",round(table(!is.na(data[data$outcome==FALSE,9]))/sum(table(!is.na(data[data$outcome==FALSE,9])))*100,1),"%)")
#the following lines of code return the mean and SD for eci score in each group
mean(data$eci[data$outcome==TRUE])
sd(data$eci[data$outcome==TRUE])
mean(data$eci[data$outcome==FALSE])
sd(data$eci[data$outcome==FALSE])
#the following lines of code return the mean and SD for dentist density in each group
mean(data$dentists[data$outcome==TRUE],na.rm = T)
sd(data$dentists[data$outcome==TRUE],na.rm = T)
mean(data$dentists[data$outcome==FALSE],na.rm = T)
sd(data$dentists[data$outcome==FALSE],na.rm = T)
#the following lines of code return the mean and SD for median household income in each group
mean(data$income[data$outcome==TRUE],na.rm = T)
sd(data$income[data$outcome==TRUE],na.rm = T)
mean(data$income[data$outcome==FALSE],na.rm = T)
sd(data$income[data$outcome==FALSE],na.rm = T)

### ICC to evaluate the need for multilevel model

In [ ]:
fipsincome = read.csv("zipincome.csv") #load median household incomes by fips code
data$countyincome = fipsincome$income[match(data$fips,fipsincome$fips)] #match county-level income to simplify model at two levels: individuals and counties (no zip codes)
emptymod = glmer(outcome_ ~ ( 1 | fips), data=data, family ="binomial") #build empty model clustered by county
emptymod@theta[1]^2/ (emptymod@theta[1]^2 + (3.14159^2/3)) #run the ICC analysis: given the result was 0.088 < 0.1, will run one-level multivariable logistic regression analysis

### Logistic regression analysis

In [ ]:
mod = glm(outcome_~age_+gender_+rucc_+median_income_+dentist_density_+dental_care_+hospital_admissions_+ECI_,data=data,family="binomial") #run the logistic regression model
a = summary(go) #save output from model
b = data.frame(a$coefficients) #save coefficients 
c = data.frame(exp(mod$coefficients)) #save odds ratios
d = data.frame(exp(confint(mod))) #save 95% CI for odds ratios
dentalresults= data.frame(cbind(b,c,d)) #save results in one data frame
colnames(dentalresults) = c("Estimate","Standard Error","Z-Value","P-Value","Odds Ratio","Confidence Inerval: 2.5","Confidence Interval: 97.5") #rename columns for results data frame
write.csv(dentalresults,"dentalresults.csv") #save results table

In [ ]:
sessionInfo()